In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import numpy as np


In [2]:
df_train = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv', index_col='id')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv', index_col='id')
df_sub = pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')

In [3]:
df_test.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
id,,,,,,,,,,,
188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [4]:
df_train.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [5]:
df_sub.head()

,id,price
0,188533,43878.016
1,188534,43878.016
2,188535,43878.016
3,188536,43878.016
4,188537,43878.016


price : target variable you want to predict, which is the sale price of the used car.

brand : The manufacturer of the car. ex) BMW, Audi, Chevrolet, etc.

model : The model name of the car. ex) BMW 328i, Chevrolet Camaro 1LT, etc.

model_year : The year the car was manufactured and rolled out from the factory. Indicates the actual production year of the vehicle.

milage : The mileage of the car, indicating how far the car has been driven.

fuel_type : The type of fuel the car uses. ex) Gasoline, Diesel, Electric, etc.

engine : THe specifications of the car’s engine. ex) displacement, number of cylinders, and horsepower.

transmission : The type of transmission in the car. ex) Automatic (A/T), Manual (M/T), etc.

ext_col : The exterior color of the car.

int_col : The interior color of the car.

accident : Indicates whether the car has been involved in any accidents or has sustained damage.

clean_title : Indicates the status of the car's title. 'Yes' / 'No' means the car has / doesn't have a clean title.


In [6]:
print('The data type of each columns of train dataset : \n')
df_train.info()

The data type of each columns of train dataset : 

<class 'pandas.core.frame.DataFrame'>
Index: 188533 entries, 0 to 188532
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   brand         188533 non-null  object
 1   model         188533 non-null  object
 2   model_year    188533 non-null  int64 
 3   milage        188533 non-null  int64 
 4   fuel_type     183450 non-null  object
 5   engine        188533 non-null  object
 6   transmission  188533 non-null  object
 7   ext_col       188533 non-null  object
 8   int_col       188533 non-null  object
 9   accident      186081 non-null  object
 10  clean_title   167114 non-null  object
 11  price         188533 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 18.7+ MB


In [7]:
features = ['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title', 'price']
num_features = ['model_year', 'milage']
cat_features = ['clean_title', 'accident', 'model', 'transmission', 'engine', 'ext_col', 'fuel_type', 'int_col', 'brand']

print('features :', features)
print('num_features :', num_features)
print('cat_features :', cat_features)

features : ['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title', 'price']
num_features : ['model_year', 'milage']
cat_features : ['clean_title', 'accident', 'model', 'transmission', 'engine', 'ext_col', 'fuel_type', 'int_col', 'brand']


In [8]:
print('The missing values of train dataset : \n', df_train.isnull().sum())
print('The missing values of test dataset : \n' , df_test.isnull().sum())

The missing values of train dataset : 
 brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64
The missing values of test dataset : 
 brand               0
model               0
model_year          0
milage              0
fuel_type        3383
engine              0
transmission        0
ext_col             0
int_col             0
accident         1632
clean_title     14239
dtype: int64


In [9]:
def cleaning_data(df):

    fuel_type_imputer = SimpleImputer(strategy='most_frequent')
    df['fuel_type'] = fuel_type_imputer.fit_transform(df[['fuel_type']]).ravel() 
    
    missing_label_imputer = SimpleImputer(strategy='constant', fill_value='missing')
    df[['accident', 'clean_title']] = missing_label_imputer.fit_transform(df[['accident', 'clean_title']])
    
    return df

In [10]:
df_train_cleaned = cleaning_data(df_train)
print('The missing valuse of train dataset after handling missing values : \n', df_train_cleaned.isnull().sum())

The missing valuse of train dataset after handling missing values : 
 brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
price           0
dtype: int64


In [11]:
# Define feature columns
num_features = ['milage']
cat_features = ['clean_title', 'accident', 'model', 'transmission', 'engine', 'ext_col', 'fuel_type', 'int_col', 'brand', 'model_year']


In [12]:
# Create preprocessing pipelines
num_pipeline = Pipeline([
    ('scaler', StandardScaler()),
])
cat_pipeline = Pipeline([
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
])

In [13]:
# Combine preprocessing pipelines
preprocessing = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features),
])

In [14]:
# Separate features and target
X_train = df_train.drop(columns=['price'])
y_train = df_train['price']
X_test = df_test

In [15]:
from sklearn.model_selection import train_test_split

# Split the training data into training and validation sets
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=1
)

In [16]:
# Define the full pipeline with preprocessing and model
full_pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('model', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=1))
])

In [17]:
# Train the model
full_pipeline.fit(X_train_split, y_train_split)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['milage']),
                                                 ('cat',
                                                  Pipeline(steps=[('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))]),
                                                  ['clean_title', 'accident',
                                                   'model', 'transmission',
                                                   'engine', 'ext_col',
                                                   'fuel_type', 'int_col',
                                                   'brand', 'model_year'])])),
                ('model', RandomForestRegressor(max_depth=10, random_state=1))])

In [18]:
# Make predictions on the validation set
val_predictions = full_pipeline.predict(X_val)

In [19]:
# Compute RMSE on the validation set
rmse = np.sqrt(mean_squared_error(y_val, val_predictions))
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 64404.627186787584


In [20]:
df_test.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
id,,,,,,,,,,,
188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [21]:
# Make predictions on the test set
test_predictions = full_pipeline.predict(X_test)

# Round the predictions to two decimal places because its a price
#test_predictions = np.round(test_predictions, 2)

# Create the submission DataFrame
submission = pd.DataFrame({
    'id': df_test.index,
    'price': test_predictions
})

# Save to CSV
submission.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")


Your submission was successfully saved!


In [22]:
submission.head()

,id,price
0,188533,16432.509652
1,188534,79462.542481
2,188535,57250.867844
3,188536,22161.364699
4,188537,30213.826731
